## 简介
- implement a fuzzy match score for n-gram precisions in the BLEU metric
- introduce two ways of extending the WORD2VEC approach to learn n-gram embedding.

## BLEU家族
部分内容来自寒小阳的博客
#### BLEU metric(Papineni et al. 2002)
BLEU: Bilingual Evaluation understudy. BLEU(Bilingual Evaluation understudy)方法由IBM提出，这种方法认为如果熟译系统魏译文越接近人工翻翻译结果，那么它的翻译质量越高。所以，评测关键就在于如何定义系统译文与参考译文之间的相似度。BLEU采用的方式是比较并统计共现的n元词的个数，即统计同时出现在系统译文和参考译文中的n元词的个数，最后把匹配到的n元词的数目除以系统译文的单词数目，得到评测结果。<br>
最开始提出的BLEU法虽然简单易行，但是它没有考虑到翻译的召回率。<br>
后对BLEU做了修正，首先计算出一个n元词在一个句子中最大可能出现的次数MaxRefCount(n-gram)，然后跟候选译文中的这个n元词出现的次数作比较，取它们之间最小值作为这个n元词的最终匹配个数。公式如下：其中Count(n-gram)是某个n元词在候选译文中的出现次数，而MaxRefCount(n-gram)是该n元词在参考译文中出现的最大次数<br>
$Count_{clip}(n-gram)=min{Count(n-gram), MaxRefCount(n-gram)}$<br>
![BLEU](https://raw.githubusercontent.com/HuangYiran/readPaper/master/fotos/BLEU.png)
one of the factors of the score is a geometric mean of n-gram precisions between the translation hypothesis and reference(s) and as a result sentences without 4-gram matches get a score of 0, even if there are good unigram, bigram and possibly trigram matches.
#### Criticism of BLEU(Callison-Burch et al. 2006)
？？？

#### NIST(Doddington, 2002)
NIST: National Institute of standards and Technology.它并不是简单的将匹配的n—gram片段数目累加起来，而是求出每个n-gram的信息量(information)，然后累加起来再除以整个译文的n-gram片段数目。信息量的计算公式是：其中分母是n元词在参考译文中出现的次数，分子是对应的n-l元词在参考译文中的出现次数。对于一元词汇，分子的取值就是整个参考译文的长度。
![NIST information](https://raw.githubusercontent.com/HuangYiran/readPaper/master/fotos/NIST_info.png)
计算信息量之后，就可以对每一个共现n元词乘以它的信息量权重，再进行加权求平均得出最后的评分结果：
![NIST score](https://raw.githubusercontent.com/HuangYiran/readPaper/master/fotos/NIST_score.png)
NIST采用的是算术平均方式，Lsys是使系统译文的长度，而Lref是参考译文的平均长度，是一个经验阈值，它使整个惩罚值在系统译文的长度是参考译文长度的2/3时为0.5。<br>
soften BLEU
#### METEOR(Banerjee and Lavie, 2005)
allowing for partial matches using lemmatization and synonyms<br>
METEOR测度的目的是解决一些BLEU标准中固有的缺陷。METEOR标准基于单精度的加权调和平均数和单字召回率。该标准是2004年Lavir发现在评价指标中召回率的意义后被提出的。他们的研究表明，召回率基础上的标准相比于那些单纯基于精度的标准(例如BLEU和NIST)，其结果和人工判断的结果有较高相关性。<br>
METEOR也包括其他指标没有发现一些其他功能，如同义词匹配，而不是只在确切的词形式匹配，匹配度量也对同义词。例如，“好”的参考渲染为“好”的翻译是一个比赛。度量也包括词干分析器，它lemmatises在lemmatised形式的话，比赛。度量标准的实施，是模块化的，这场比赛的话作为模块实现的算法，以及新的模块，实现不同的匹配策略可以很容易地添加。
#### LEBLEU(Virpioja and Groenroos, 2015)
where BLEUs augmented with fuzzy matches based on the character-level Levenstein distance.<br>
莱文斯坦距离，又称Levenshtein距离，是编辑距离的一种。指两个字串之間，由一个转成另一个所需的最少编辑操作次数。允许的编辑操作包括将一个字符替换成另一个字符，插入一个字符，刪除一个字符。<br>
directly implementing fuzzy matches between n-grams<br>
#### CHRF(Popovic, 2015)
？？？

## Embedding
考虑n-gram embedding，使得他们能够表达短语之间的相似性，比如hate和do not like应该具有相似的意思。
#### Separate N-gram Embeddings
- Our first approach is learning separate embedding models for unigrams, bigrams and trigrams. 
- While unigram embeddings are handled by the baseline WORD2VEC method, in this approach we group the n-gram tokens into a single entry(即把n个词当成一个整体来对待), ignoring the overlapping parts.
- and then compute embeddings for the new tokens with the baseline approach.
- since then umber of different n-grams is much highter than single tokens, we filter out bi-grams that occur less than 30 times and tri-grams that occur less than 50 times.
#### Joint N-gram Embeddings
- 上一个方法只能学习相同长度的短语之间的相似性。while it is enough for this submission's metric, it also runs the danger of learning overlapping n-grams, as these are generated next to each other. 
- skip gram training for words and n-grams
![skip gram training forwords and n-grams](https://raw.githubusercontent.com/HuangYiran/readPaper/master/fotos/Joint_n_gram_embeddings.png)
- In addition to frequency filtering we also sample the n-grams randomly, sometimes including or excluding them from training. To increase the chances of more rare n-grams beging included we define the sampling probability based on smoothed reverse frequency(还有这个功能啊？？)：$p = exp(-\beta log(f))=\frac{1}{f^{\beta}}$
- while f is the n-gram absolute frequency, p is the sampling probability and $\beta$ is a small weight. Using this dynamic probability is equivalent to down-sampling the more frequent n-grams, leaving more exposure to the entries with lower frequency
- finally, be sampling only n-grams that do not overlap we reduce the problem to the original word-level WORD2VEC randomly re-deciding which n-grams to join into a single lexical entry at each epoch(这个不是很懂啊？？？). This also means that n-grams are present as both the input and output entries.

## BLEU2VEC
相较与原BLEU，做了一下修改
1. 相同的词权重为1，（即原BLEU）
2. 类似的词（embedding）根据cos相识度计算权值，优先挑选相似度高的对。
3. 重叠是禁止的。overlaps are not allowed: once a pair is aligned it is removed from the search space for the next n-grams

The rationale behind this modification is that partially correct words will e hopefully considered similar by the embedding model, while completely wrong words will only find alignments with lower similarity.

## 注意？？？
- key 用MT译文，Query用reference，做attention。
- 进行单词或者短语embedding（看一下stanford的recurrent NN，另外需要更好的理解微软的基于短语的MT）
- 有没有其他方法，可以找到两个句子中互相对应的地方。想一下微软他们家的对应方法，再想一下sequence transduction他们家的对应方法
- 想想能否结合recurrent NN和embedding？？